In [103]:
import random
random.seed(42) # define the seed (important to reproduce the results)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
data = pd.read_csv('../dataset/breast-cancer.csv', sep = ';')
data = data.dropna(axis='rows') #remove NaN


In [ ]:
print("Número de linhas e colunas na matriz de atributos:", data.shape)
attributes = list(data.columns)
data.head(10) # showing the first ten lines 

Vamos remover as linhas que contenham "?" ou qualquer outro caracter que indique um erro ou entrada faltante.

Primeiramente, convertemos essas entradas para "NaN":

In [106]:
# Já que nem todos os atributos são numéricos, vou convertê-los primeiro
# training_data = data.to_numpy()
# nrow,ncol = training_data.shape
# y = training_data[:,-1]
# X = training_data[:,0:ncol-1]

In [107]:
import numpy as np
# Substitui Nan por um caracter desejado
data = data.replace('?', np.nan) # presente em dois atributos da base 'breast-cancer'

Apagando as linhas com NaN

In [108]:
# remove as linhas com NaN
data = data.dropna() 

## Binarização dos dados

In [109]:
training_data = data.iloc[:, 0:9].values

In [110]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder_NodeCaps = LabelEncoder()
label_encoder_Breast = LabelEncoder()
label_encoder_Irradiat = LabelEncoder()

In [111]:
training_data[:,4] = label_encoder_NodeCaps.fit_transform(training_data[:,4])
training_data[:,6] = label_encoder_Breast.fit_transform(training_data[:,6])
training_data[:,8] = label_encoder_Irradiat.fit_transform(training_data[:,8])

| OnehotEncoder - Converter valores nominais e ordinais para inteiro 

**Contando quantas opções de resposta tem um determinado atributo**

In [112]:
len(np.unique(data['node-caps'])) # atributo inicialmente com um valor '?'

2

In [113]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [114]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,1,2,3,5,7])])

In [115]:
training_data = onehotencoder.fit_transform(training_data)

In [122]:
training_data.shape

array(['50-59', 'ge40', '15-19', '0-2', 'no', 1, 'right', 'central', 'no',
       'no-recurrence-events'], dtype=object)

In [117]:
# nrow,ncol = x_prev.shape
# armazeno os valores do atributo alvo (class)
# y_class = data.iloc[:, 9].values 

In [118]:
training_data = data.to_numpy()
nrow,ncol = training_data.shape
y = training_data[:,-1]
X = training_data[:,0:ncol-1]

Selecionando os conjuntos de treinamento e teste.

In [119]:
from sklearn.model_selection import train_test_split
p = 0.7 # fracao de elementos no conjunto de treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

Classificação usando a biblioteca scikit-learn

In [120]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score

model = GaussianNB()
model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# score = accuracy_score(y_pred, y_test)
# print('Accuracy:', score)

ValueError: could not convert string to float: '30-39'